In [1]:
import pandas as pd
import random
import numpy as np
import os

# Configuração da semente para os dados serem sempre os mesmos (reprodutibilidade)
random.seed(42)
np.random.seed(42)

# 1. Definição das Categorias e Produtos típicos de mercado
produtos_base = {
    'Açougue': ['Alcatra', 'Picanha', 'Carne Moída', 'Frango Inteiro', 'Costela Suína', 'Linguiça Toscana', 'Coxão Mole', 'Filé Mignon'],
    
    'Mercearia': ['Arroz 5kg', 'Feijão 1kg', 'Açúcar 1kg', 'Óleo de Soja', 'Farinha de Trigo', 'Café 500g', 'Biscoito Recheado', 'Molho de Tomate', 'Salgadinhos', 'Milho em Conserva', 'Ervilha', 'Azeitona', 'Fermento'],
    
    'Bebidas': ['Cerveja Lata 350ml', 'Refrigerante 2L', 'Suco Caixa 1L', 'Água Mineral 500ml', 'Vinho Tinto', 'Vodka', 'Whisky'],
    
    'Hortifruti': ['Batata kg', 'Tomate kg', 'Banana kg', 'Maçã kg', 'Alface', 'Cebola kg', 'Cenoura kg', 'Laranja kg', 'Ovos Dúzia'],
    
    'Higiene/Limpeza': ['Sabão em Pó 1kg', 'Detergente', 'Papel Higiênico (12 rolos)', 'Shampoo', 'Pasta de Dente', 'Amaciante', 'Água Sanitária', 'Desinfetante', 'Papel Toalha', 'Sabão em Barra'],
    
    'Padaria': ['Pão Francês kg', 'Leite Integral 1L', 'Queijo Mussarela kg', 'Presunto kg', 'Bolo Industrializado', 'Pão de Forma', 'Margarina', 'Requeijão', 'Iogurte', 'Manteiga', 'Maionese'],
    
    'Bazar': ['Balde Plástico', 'Bacia', 'Vassoura', 'Rodo', 'Saco de Lixo 30L', 'Saco de Lixo 100L', 'Esponja de Louça', 'Lã de Aço', 'Lâmpada LED', 'Pilha AA', 'Pote de Vidro', 'Pote Plástico', 'Chinelo', 'Pá de Lixo', 'Vela'],
    
    'Condimentos': ['Azeite de Oliva 500ml', 'Vinagre', 'Catchup', 'Mostarda', 'Molho de Pimenta', 'Sal Refinado 1kg'],
    
    'Congelados': ['Pizza Calabresa', 'Pizza Mussarela', 'Sorvete 2L', 'Lasanha Congelada', 'Hambúrguer Caixa'],
    
    'Doces/Sobremesa': ['Chocolate Barra', 'Achocolatado em Pó', 'Gelatina', 'Leite Condensado', 'Creme de Leite'],
    
    'Diet/Light': ['Adoçante Líquido', 'Adoçante Sachê']
}

marcas = ['Econômica', 'Marca MC (Própria)', 'Premium', 'Líder de Mercado']

lista_produtos = []




In [2]:
# 2. Gerar os dados
id_contador = 100  # Começando SKUs no 100

print("Gerando catálogo de produtos.\n")

for categoria, itens in produtos_base.items():
    for item in itens:
        for marca in marcas:
            # LÓGICA DE PRECIFICAÇÃO INTELIGENTE            
            fator_marca = 1.0
            if marca == 'Econômica': fator_marca = 0.85 
            if marca == 'Premium': fator_marca = 1.40   
            if marca == 'Marca MC (Própria)': fator_marca = 0.90 
            
            # Custo Base Aleatório (R$)
            custo_base = round(random.uniform(2.50, 25.0) * fator_marca, 2)
            
            # AJUSTES ESPECÍFICOS POR CATEGORIA
            
            # Bazar têm margens altas, mas custo variado
            if categoria == 'Bazar': 
                custo_base = round(random.uniform(5.0, 40.0), 2)
            
            # Açougue e Congelados são caros
            if categoria in ['Açougue', 'Congelados']: 
                custo_base = round(custo_base * 1.8 + 5, 2)
            
            # Bebidas Alcoólicas caras
            if 'Whisky' in item or 'Vodka' in item: 
                custo_base = round(custo_base * 3.0, 2)
            
            # Hortifruti e Condimentos baratos
            if categoria in ['Hortifruti', 'Condimentos', 'Diet/Light']: 
                custo_base = round(custo_base * 0.4 + 1, 2)
                
            # Azeite é exceção (caro)
            if 'Azeite' in item: custo_base += 15.0
            
            # MARGENS DE LUCRO (Mark-up)
            # MC Supermercado (Varejo) -> Margem maior (serviço, conveniência)
            # MC Atacadista (Atacado) -> Margem menor (ganho no volume)
            
            margem_varejo = random.uniform(1.30, 1.50) # 30% a 50%
            margem_atacado = random.uniform(1.10, 1.20) # 10% a 20%
            
            # Bazar tem margem altíssima no varejo (até 80%)
            if categoria == 'Bazar': margem_varejo = random.uniform(1.60, 1.80)

            preco_sup = round(custo_base * margem_varejo, 2)
            preco_atac = round(custo_base * margem_atacado, 2)
            
            lista_produtos.append({
                'SKU': id_contador,
                'Produto': f"{item} - {marca}",
                'Categoria': categoria,
                'Custo_Unitario': custo_base,
                'Preco_Venda_Sup': preco_sup,
                'Preco_Venda_Atac': preco_atac
            })
            id_contador += 1
        

Gerando catálogo de produtos.



In [3]:
# 3. Criar e Padronizar DataFrame
df_produtos = pd.DataFrame(lista_produtos)

def padronizar_dados_para_bd(df_input):
    """
    Função para limpar metadados e padronizar o DataFrame para inserção em Banco de Dados.
    
    Ações:
    1. Remove espaços extras de colunas de texto (strip).
    2. Padroniza nomes das colunas para Snake Case (minusculo_com_underline).
    3. Garante que colunas numéricas tenham a tipagem correta.
    4. Remove duplicatas acidentais.
    """
    # Cria uma cópia para não alterar o original sem querer
    df = df_input.copy()
    
    # 1. Limpeza de Strings (remove espaços vazios no início/fim)
    # Itera apenas sobre colunas do tipo 'object' (texto)
    colunas_texto = df.select_dtypes(include=['object']).columns
    for col in colunas_texto:
        df[col] = df[col].str.strip()
    
    # 2. Padronização de Colunas (Snake Case)
    # Ex: 'Custo_Unitario' vira 'custo_unitario'
    # Isso facilita muito consultas SQL futuras
    df.columns = (df.columns
                  .str.lower()            # Tudo para minúsculo
                  .str.replace(' ', '_')  # Espaços viram underline
                  .str.replace('/', '_')  # Barras viram underline
                  .str.normalize('NFKD')  # Remove acentuação (opcional, mas recomendado para nomes de colunas)
                  .str.encode('ascii', errors='ignore')
                  .str.decode('utf-8')
    )
    
    # 3. Garantia de Tipagem Numérica
    # Força as colunas de preço a serem float arredondados em 2 casas
    colunas_numericas = df.select_dtypes(include=['number']).columns
    for col in colunas_numericas:
        if col in df.columns and col != 'sku':
            df[col] = df[col].astype(float).round(2)
            
    # 4. Remove duplicatas (se houver linhas idênticas)
    df = df.drop_duplicates()
    
    return df


# Aplique a função no seu DataFrame original
df_produtos_limpo = padronizar_dados_para_bd(df_produtos)

# Visualizar a diferença nos nomes das colunas
print("Colunas antigas:", df_produtos.columns.tolist())
print("Colunas novas:  ", df_produtos_limpo.columns.tolist())

# Atualize a variável principal para salvar o CSV correto
df_produtos = df_produtos_limpo

Colunas antigas: ['SKU', 'Produto', 'Categoria', 'Custo_Unitario', 'Preco_Venda_Sup', 'Preco_Venda_Atac']
Colunas novas:   ['sku', 'produto', 'categoria', 'custo_unitario', 'preco_venda_sup', 'preco_venda_atac']


In [4]:
# 4. Visualizar os primeiros itens
print("Amostra da Tabela de Produtos Gerada:")
print(df_produtos.head(10))

Amostra da Tabela de Produtos Gerada:
   sku                           produto categoria  custo_unitario  \
0  100               Alcatra - Econômica   Açougue           30.83   
1  101      Alcatra - Marca MC (Própria)   Açougue           17.19   
2  102                 Alcatra - Premium   Açougue           61.88   
3  103        Alcatra - Líder de Mercado   Açougue           10.71   
4  104               Picanha - Econômica   Açougue            9.73   
5  105      Picanha - Marca MC (Própria)   Açougue           28.92   
6  106                 Picanha - Premium   Açougue           57.20   
7  107        Picanha - Líder de Mercado   Açougue           37.78   
8  108           Carne Moída - Econômica   Açougue           41.77   
9  109  Carne Moída - Marca MC (Própria)   Açougue           12.58   

   preco_venda_sup  preco_venda_atac  
0            40.23             34.76  
1            24.88             20.07  
2            81.52             70.68  
3            14.39             12.3

In [5]:
# 5. Salvar em CSV
caminho_pasta = r'D:\Projetos\MC_Supermercados' 

# Verifica se a pasta existe, se não, salva na pasta atual mesmo e avisa
if os.path.exists(caminho_pasta):
    caminho_completo = os.path.join(caminho_pasta, 'dim_produtos.csv')
    df_produtos.to_csv(caminho_completo, index=False, encoding='utf-8-sig')
    print(f"Arquivo salvo com sucesso em: {caminho_completo}\n")
else:
    # Se você não definiu o caminho acima, ele salva na pasta atual e te diz onde é
    df_produtos.to_csv('dim_produtos.csv', index=False, encoding='utf-8-sig')
    print("Pasta específica não encontrada ou não definida.")
    print(f"O arquivo foi salvo na pasta padrão do Python: {os.getcwd()}")
    print("Nome do arquivo: dim_produtos.csv")

print(f"✅ SUCESSO! Base atualizada gerada com {len(df_produtos)} produtos.")



Arquivo salvo com sucesso em: D:\Projetos\MC_Supermercados\dim_produtos.csv

✅ SUCESSO! Base atualizada gerada com 364 produtos.
